In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
# ! pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import geocoder 
import math
print('Libraries imported.')

Libraries imported.


In [2]:
# CLIENT_ID = 'YA1V0ZT4RLWE3PVLWULDI0QLPHCUNN0BNXSOT5GS3P1EZR0G' # your Foursquare ID
# CLIENT_SECRET = 'ITTIADOFH3REMVNUNONWLMRG0T2051ETQD5GCDGDXBWNRJY5' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version
# LIMIT = 100 # A default Foursquare API limit value

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YA1V0ZT4RLWE3PVLWULDI0QLPHCUNN0BNXSOT5GS3P1EZR0G
CLIENT_SECRET:ITTIADOFH3REMVNUNONWLMRG0T2051ETQD5GCDGDXBWNRJY5


In [3]:
path = "/Users/isalau/Documents/GitHub/Coursera_Capstone/Neighborhood_Labels.csv"
dc = pd.read_csv(path)  

In [4]:
dc = dc.rename({'X': 'Longitude'}, axis='columns')
dc = dc.rename({'Y': 'Latitude'}, axis='columns')
dc = dc.rename({'NAME': 'Neighborhood'}, axis='columns')

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
path = "/Users/isalau/Documents/GitHub/Coursera_Capstone/dc_venues.csv"
dc_venues_from_csv= pd.read_csv(path)
dc_venues_from_csv.head()
dc_venues = dc_venues_from_csv

In [7]:
# dc_venues = getNearbyVenues(names=dc['Neighborhood'],
#                                    latitudes=dc['Latitude'],
#                                    longitudes=dc['Longitude']
#                                   )

In [8]:
categories = dc_venues['Venue Category'].to_frame()


In [9]:
categories.insert(1, 'Venue Count',0)


In [10]:
categories_count = categories.groupby('Venue Category').count()

In [11]:
categories_count.sort_values(by=['Venue Count'], ascending = False)

,Venue Count
Venue Category,
Coffee Shop,94
Pizza Place,71
Hotel,69
Sandwich Place,62
American Restaurant,56
Park,56
Convenience Store,52
Liquor Store,52
Grocery Store,45


In [12]:
dc_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
16th Street Heights,14,14,14,14,14,14
Adams Morgan,57,57,57,57,57,57
American University Park,2,2,2,2,2,2
Arboretum,15,15,15,15,15,15
Barnaby Woods,4,4,4,4,4,4
Barry Farm,6,6,6,6,6,6
Bellevue,5,5,5,5,5,5
Benning,16,16,16,16,16,16
Benning Ridge,4,4,4,4,4,4


In [13]:
print('There are {} uniques categories.'.format(len(dc_venues['Venue Category'].unique())))

There are 308 uniques categories.


In [14]:
# one hot encoding
dc_onehot = pd.get_dummies(dc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dc_onehot['Neighborhood'] = dc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dc_onehot.columns[-1]] + list(dc_onehot.columns[:-1])
dc_onehot = dc_onehot[fixed_columns]

In [15]:
dc_grouped = dc_onehot.groupby('Neighborhood').mean().reset_index()

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dc_grouped['Neighborhood']

for ind in np.arange(dc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dc_grouped.iloc[ind, :], num_top_venues)


In [23]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,16th Street Heights,Coffee Shop,Salvadoran Restaurant,Breakfast Spot,Bus Stop,Chinese Restaurant,Park,Bed & Breakfast,Greek Restaurant,Gym,Gymnastics Gym
1,0,Adams Morgan,Diner,Mediterranean Restaurant,Ethiopian Restaurant,Bar,Spa,Asian Restaurant,Ice Cream Shop,Coffee Shop,Cocktail Bar,Brazilian Restaurant
2,0,American University Park,Italian Restaurant,BBQ Joint,Yoga Studio,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
3,0,Arboretum,Garden,Ice Cream Shop,Chinese Restaurant,Basketball Court,Fast Food Restaurant,Botanical Garden,Automotive Shop,Storage Facility,Gas Station,Nightclub
4,0,Barnaby Woods,Field,Gym / Fitness Center,Park,BBQ Joint,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
5,0,Barry Farm,Bus Stop,Basketball Court,Intersection,Metro Station,Rental Car Location,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market
6,0,Bellevue,Grocery Store,Baseball Field,Playground,Pizza Place,Shoe Repair,Farmers Market,Ethiopian Restaurant,Event Service,Event Space,Exhibit
7,0,Benning,Discount Store,Liquor Store,Restaurant,Sandwich Place,Pharmacy,Donut Shop,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Food
8,2,Benning Ridge,Shipping Store,Insurance Office,Convenience Store,Burger Joint,Field,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
9,0,Bloomingdale,Bus Stop,Yoga Studio,Grocery Store,Deli / Bodega,Convenience Store,Dive Bar,Pizza Place,Dog Run,Coffee Shop,Park


# KMeans

In [18]:
# set number of clusters
kclusters = 3

dc_grouped_clustering = dc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

ValueError: cannot insert Cluster Labels, already exists

In [25]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,16th Street Heights,Coffee Shop,Salvadoran Restaurant,Breakfast Spot,Bus Stop,Chinese Restaurant,Park,Bed & Breakfast,Greek Restaurant,Gym,Gymnastics Gym
1,0,Adams Morgan,Diner,Mediterranean Restaurant,Ethiopian Restaurant,Bar,Spa,Asian Restaurant,Ice Cream Shop,Coffee Shop,Cocktail Bar,Brazilian Restaurant
2,0,American University Park,Italian Restaurant,BBQ Joint,Yoga Studio,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
3,0,Arboretum,Garden,Ice Cream Shop,Chinese Restaurant,Basketball Court,Fast Food Restaurant,Botanical Garden,Automotive Shop,Storage Facility,Gas Station,Nightclub
4,0,Barnaby Woods,Field,Gym / Fitness Center,Park,BBQ Joint,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
5,0,Barry Farm,Bus Stop,Basketball Court,Intersection,Metro Station,Rental Car Location,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market
6,0,Bellevue,Grocery Store,Baseball Field,Playground,Pizza Place,Shoe Repair,Farmers Market,Ethiopian Restaurant,Event Service,Event Space,Exhibit
7,0,Benning,Discount Store,Liquor Store,Restaurant,Sandwich Place,Pharmacy,Donut Shop,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Food
8,2,Benning Ridge,Shipping Store,Insurance Office,Convenience Store,Burger Joint,Field,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
9,0,Bloomingdale,Bus Stop,Yoga Studio,Grocery Store,Deli / Bodega,Convenience Store,Dive Bar,Pizza Place,Dog Run,Coffee Shop,Park


In [20]:
dc_merged = dc

# merge dc_grouped with dc_data to add latitude/longitude for each neighborhood
dc_merged = dc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [21]:
address = 'Washington,DC'

geolocator = Nominatim(user_agent="dc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of DC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 38.8949924, -77.0365581.


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# rainbow= [colors.navy, colors.crimson, colors.darkviolet]
rainbow=["#35063e", "#0343df", "#c20078"]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dc_merged['Latitude'], dc_merged['Longitude'], dc_merged['Neighborhood'], dc_merged['Cluster Labels']):
    if (math.isnan(cluster)):
        clus = 3
    else: 
        clus = int(cluster)
    label = folium.Popup('Neighborhood: '+ str(poi) + ' Cluster: ' + str(clus), parse_html=True)


    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clus-1],
        fill=True,
        fill_color=rainbow[clus-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters